In [59]:
import pymysql
import pyecharts
from pyecharts import options as opts
from pyecharts.charts import Page,Bar,Map,Line,WordCloud

In [60]:
def get_cursor(sql): 
    # 连接数据库
    connect = pymysql.Connect(host='localhost', port=3306, user='root', passwd='123456', db='test_db',
                              charset='utf8')
    # 获取游标
    cursor = connect.cursor()
    cursor.execute(sql)
    data=cursor.fetchall()    
    # 关闭连接
    cursor.close()
    connect.close()
    return data


In [73]:
data=get_cursor("SELECT DISTINCT area,CAST(`value` AS SIGNED INTEGER) `value` FROM farming WHERE product='猪肉' AND item='产量' AND time='2018' AND area!='全国' ORDER BY CAST(`value` AS SIGNED INTEGER) DESC limit 10")
print(data)

(('四川', 481), ('河南', 479), ('湖南', 446), ('山东', 421), ('湖北', 333), ('云南', 323), ('河北', 286), ('广东', 281), ('广西', 263), ('江西', 246))


In [74]:
x_tag = [i[0] for i in data]
y_val=[float(i[1]) for i in data]
print(x_tag)
print(y_val)
data=list(data)
print(data)

['四川', '河南', '湖南', '山东', '湖北', '云南', '河北', '广东', '广西', '江西']
[481.0, 479.0, 446.0, 421.0, 333.0, 323.0, 286.0, 281.0, 263.0, 246.0]
[('四川', 481), ('河南', 479), ('湖南', 446), ('山东', 421), ('湖北', 333), ('云南', 323), ('河北', 286), ('广东', 281), ('广西', 263), ('江西', 246)]


In [62]:
name = x_tag
sales = y_val
bar = (
        Bar()
        .add_xaxis(x_tag)
        .add_yaxis("成交量", y_val)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="生猪产量top10柱形图"),
        )
        .reversal_axis() # 添加此句变为条形图
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max",name="最大值"),opts.MarkPointItem(type_="min",name="最小值"),])
        )
)
bar.render_notebook()

In [63]:
data1=get_cursor("SELECT DISTINCT area,CAST(`value` AS SIGNED INTEGER) `value` FROM farming WHERE product='猪肉' AND item='产量' AND time='2018' AND area!='全国' ORDER BY CAST(`value` AS SIGNED INTEGER) DESC")
print(data1)

(('四川', 481), ('河南', 479), ('湖南', 446), ('山东', 421), ('湖北', 333), ('云南', 323), ('河北', 286), ('广东', 281), ('广西', 263), ('江西', 246), ('安徽', 243), ('辽宁', 210), ('江苏', 205), ('贵州', 164), ('黑龙江', 149), ('重庆', 132), ('吉林', 127), ('福建', 113), ('陕西', 86), ('浙江', 74), ('内蒙古', 71), ('山西', 62), ('甘肃', 50), ('海南', 45), ('新疆', 38), ('天津', 21), ('北京', 13), ('上海', 11), ('青海', 9), ('宁夏', 8), ('西藏', 1))


In [64]:
data1 =list(data1)
def map_china() -> Map:
  c = (
    Map()
    .add(series_name="生猪产量(万吨)", data_pair=data1, maptype="china",zoom = 1,center=[105,38])
    .set_global_opts(
      title_opts=opts.TitleOpts(title="全国生猪产量地图"),
      visualmap_opts=opts.VisualMapOpts(max_=9999,is_piecewise=True,
              pieces=[{"max": 9, "min": 0, "label": "0-9","color":"#FFE4E1"},
                  {"max": 99, "min": 10, "label": "10-99","color":"#FF7F50"},
                  {"max": 499, "min": 100, "label": "100-499","color":"#F08080"},
                  {"max": 999, "min": 500, "label": "500-999","color":"#CD5C5C"},
                  {"max": 9999, "min": 1000, "label": ">=1000", "color":"#8B0000"}]
                       )
    )
  )
  return c
 
d_map = map_china()
d_map.render_notebook()

In [65]:
data11=get_cursor("SELECT aa.*,bb.stage FROM (SELECT DISTINCT CAST(`value` AS SIGNED INTEGER) `value`,area FROM farming WHERE item='数量'"+ 
"AND area!='全国' AND time='2018' AND product='畜禽本期出栏数_猪') aa INNER JOIN "+
"(SELECT DISTINCT CAST(`value` AS SIGNED INTEGER) stage,area FROM farming WHERE item='数量' "+
"AND area!='全国' AND time='2018' AND product='畜禽存栏数(年末数)_猪')bb ON aa.area=bb.area")
print(data11)

((169, '北京', 45), (278, '天津', 196), (3709, '河北', 1820), (814, '山西', 549), (896, '内蒙古', 497), (2495, '辽宁', 1262), (1570, '吉林', 870), (1964, '黑龙江', 1353), (148, '上海', 96), (2680, '江苏', 1552), (911, '浙江', 516), (2837, '安徽', 1356), (1421, '福建', 799), (3124, '江西', 1587), (5082, '山东', 2985), (6402, '河南', 4337), (4363, '湖北', 2521), (5993, '湖南', 3822), (3757, '广东', 2024), (3465, '广西', 2298), (561, '海南', 382), (1758, '重庆', 1167), (6638, '四川', 4258))


In [66]:
date = [i[1] for i in data11]
sales1 = [i[0] for i in data11]
sales2 = [i[2] for i in data11]
line = (
        Line()
        .add_xaxis(date)
        .add_yaxis("出栏数",sales1,
            markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average",name="平均值")]),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max",name="最大值"),opts.MarkPointItem(type_="min",name="最小值")],symbol="diamond",symbol_size=30),  
        )
        .add_yaxis(
            "存栏数",sales2,
            markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average",name="平均值")]),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max",name="最大值"),opts.MarkPointItem(type_="min",name="最小值")],),  
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="2018年生猪存栏与出栏折线图"),
            xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
            yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
        )
    )
line.render_notebook()

In [67]:
data2=get_cursor("SELECT DISTINCT time,CAST(`value` AS SIGNED INTEGER) `value` FROM farming WHERE product='猪肉' AND item='批发价' ORDER BY time")
print(data2)

(('201844', 19), ('201845', 19), ('201846', 19), ('201847', 19), ('201848', 19), ('201849', 19), ('201850', 19), ('201851', 19), ('201852', 19), ('201901', 25), ('201902', 19), ('201903', 18), ('201904', 18), ('201905', 17), ('201906', 18), ('201907', 18), ('201908', 18), ('201909', 18), ('201910', 18), ('201911', 19), ('201912', 20), ('201913', 20), ('201914', 20), ('201915', 20), ('201916', 20), ('201917', 20), ('201918', 20), ('201919', 20), ('201920', 20), ('201921', 20), ('201922', 20), ('201923', 21), ('201924', 21), ('201925', 21), ('201926', 22), ('201927', 22), ('201928', 23), ('201929', 23), ('201930', 24), ('201931', 24), ('201932', 25), ('201933', 27), ('201934', 30), ('201935', 33), ('201936', 34), ('201937', 35), ('201938', 36), ('201939', 36), ('201940', 36), ('201941', 40), ('201942', 45), ('201943', 50), ('201944', 52), ('201945', 51), ('201946', 48), ('201947', 43), ('201948', 42), ('201949', 43), ('201950', 43), ('201951', 43), ('201952', 43), ('202001', 44), ('20200

In [68]:
x_tag2 = [i[0] for i in data2]
y_val2=[i[1] for i in data2]
line = (
        Line()
        .add_xaxis(x_tag2)
        .add_yaxis("批发价",y_val2,
            markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average",name="平均值")]),
            markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max",name="最大值"),opts.MarkPointItem(type_="min",name="最小值")],symbol="diamond",symbol_size=30),  
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="全国平均猪肉价格走势"),
            xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
            yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
        )
    )
line.render_notebook()

In [69]:
data3=get_cursor("SELECT item,count(*) FROM farming GROUP BY item")
print(data3)

(('产量', 7309), ('容积', 10), ('容量', 22), ('批发价', 500), ('拥有量', 87), ('播种面积', 1618), ('数量', 2110), ('金额', 28), ('销售量', 220), ('销售额', 232), ('面积', 988))


In [70]:
keyword = [i[0] for i in data3]
count = [i[1] for i in data3]
wordcloud = (
        WordCloud()
        .add("", [(k,c) for k,c in zip(keyword,count)] ,
             shape='star', rotate_step=20,
            )
        .set_global_opts(title_opts=opts.TitleOpts(title="农业数据标签统计"))
)
wordcloud.render_notebook()

In [71]:
data4=get_cursor("SELECT product,count(*) FROM farming GROUP BY product")
print(data4)

(('中国兰花', 12), ('中国兰花种植', 6), ('中稻和一季晚稻', 73), ('主要盆景', 12), ('主要盆景种植', 6), ('主要盆栽植物', 14), ('主要盆栽植物种植', 7), ('主要鲜切花', 14), ('主要鲜切花种植', 7), ('亚麻', 20), ('人均棉花', 42), ('人均水产品', 110), ('人均油料', 48), ('人均粮食', 46), ('人均糖料', 10), ('传统渔民', 15), ('全年屠宰', 20), ('兰花类', 14), ('兰花类种植', 7), ('其他农作物', 10), ('其他茶', 13), ('其他谷物', 74), ('其它园林水果', 30), ('养殖_其他类', 7), ('养殖_甲壳类', 7), ('养殖_藻类', 7), ('养殖_贝类', 7), ('养殖_鱼类', 7), ('养殖水产品', 109), ('冬小麦', 68), ('凤梨类', 14), ('凤梨类种植', 7), ('剑麻番麻(折纤维)', 7), ('剑麻番麻(折纤维)实有', 7), ('剑麻番麻(折纤维)收获', 7), ('剑麻番麻(折纤维)新植', 7), ('半细羊毛', 13), ('南美白对虾', 9), ('双季晚稻', 16), ('向日葵籽', 50), ('咖啡豆(按干咖啡豆计算)', 7), ('咖啡豆(按干咖啡豆计算)实有', 7), ('咖啡豆(按干咖啡豆计算)收获', 7), ('咖啡豆(按干咖啡豆计算)新植', 7), ('唐菖蒲', 14), ('唐菖蒲种植', 7), ('园林水果', 30), ('国光苹果', 30), ('夏粮', 88), ('大豆', 95), ('大麦', 34), ('大麻(线麻)', 20), ('大黄鱼', 7), ('奶类', 146), ('家禽出栏', 69), ('小型屠宰点屠宰', 18), ('小麦', 106), ('屠宰企业', 20), ('山羊毛', 13), ('工业及其他用途花卉', 14), ('工业及其他用途花卉种植', 7), ('干燥花', 7), ('干燥花种植', 6), ('扣蟹', 9), ('投放鱼种', 10), ('捕捞', 46), ('捕捞_其

In [72]:
keyword1 = [i[0] for i in data4]
count1 = [i[1] for i in data4]
wordcloud = (
        WordCloud()
        .add("", [(k,c) for k,c in zip(keyword1,count1)] ,
             shape='star', rotate_step=20,
            )
        .set_global_opts(title_opts=opts.TitleOpts(title="农产品名词统计"))
)
wordcloud.render_notebook()